# MomsHelperAI - Intelligent Planning helper Multi-Agent System Demo

**A multi-agent system where specialised AI agents collaborate to automate the entire family planning workflow from meal discovery to shopping list optimisation.**

This notebook demonstrates a production-ready multi-agent system using:
- Google Agent Development Kit (ADK)
- Gemini 2.5 Flash Lite LLM
- SQLite for data persistence

Configuration section

In [ ]:
import sys
import subprocess

subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt", "-q"])
print("Installation complete")

In [ ]:
# Configure environment
import os
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()
print(f"API Key: {os.getenv('GOOGLE_API_KEY', '')[:10]}...")

## 1. Setup and Imports

In [ ]:
import asyncio
import os
from datetime import datetime, timedelta
import importlib
import sys

# Force reload modules to get latest changes
if 'agents.meal_planner' in sys.modules:
    importlib.reload(sys.modules['agents.base_agent'])
    importlib.reload(sys.modules['agents.search_agent'])
    importlib.reload(sys.modules['agents.meal_planner'])
    importlib.reload(sys.modules['agents.week_planner'])
    importlib.reload(sys.modules['agents.grocery_planner'])
    importlib.reload(sys.modules['agents.orchestrator'])
    print("Reloaded agent modules")

# MomsHelperAI imports
from agents.orchestrator import orchestrator
from agents.meal_planner import meal_planner_agent
from agents.week_planner import week_planner_agent
from agents.grocery_planner import grocery_planner_agent
from storage.sqlite_storage import SQLiteStorage
from utils.config import Config

print("All imports successful")
print(f"API Key configured: {Config.GOOGLE_API_KEY[:10]}...")
print(f"Using Gemini model: gemini-2.5-flash-lite")

## 2. Initialize Storage and Load Sample Data

In [ ]:
# Initialize storage
storage = SQLiteStorage()

# Load sample Sharma family
sharma_family = {
    'id': 'sharma_001',
    'name': 'Sharma Family',
    'members': [
        {'name': 'Rajesh', 'age': 38, 'role': 'father'},
        {'name': 'Priya', 'age': 35, 'role': 'mother'},
        {'name': 'Aarav', 'age': 10, 'role': 'son'},
        {'name': 'Ananya', 'age': 7, 'role': 'daughter'}
    ],
    'dietary_restrictions': ['vegetarian'],
    'preferred_cuisines': ['North Indian', 'South Indian', 'Gujarati'],
    'allergies': [],
    'spice_level': 'medium'
}

gonzalez_family = {
    'id': 'gonzalez_001',
    'name': 'Gonzalez Family',
    'members': [
        {'name': 'Carlos', 'age': 42, 'role': 'father'},
        {'name': 'Maria', 'age': 39, 'role': 'mother'},
        {'name': 'Luis', 'age': 15, 'role': 'son'},
        {'name': 'Camila', 'age': 13, 'role': 'daughter'},
        {'name': 'Mateo', 'age': 7, 'role': 'son'}
    ],
    'dietary_restrictions': ['no_pork'],
    'preferred_cuisines': ['Mexican', 'Spanish', 'Tex-Mex'],
    'allergies': ['peanuts'],
    'spice_level': 'high'
}

try:
    storage.create_family(sharma_family)   
    print("Sharma family data loaded")
except:
    print("Sharma family already exists")

try:
    storage.create_family(gonzalez_family)   
    print("gonzalez family data loaded")
except:
    print("gonzalez family already exists")

## 3. Demo 1: Meal Planning with MealPlannerAgent

**This demonstrates:**
- MealPlannerAgent using google_search for web recipes
- Real LLM response from Gemini

### Important: API Rate Limits

**Free tier limits for `gemini-2.5-flash-lite`:**
- 15 requests per minute
- Each meal plan uses ~5-10 API calls
- **Wait 60 seconds between runs if you get quota errors**

If you see "quota exceeded" error:
1. Wait 1 minute
2. Re-run the cell
3. Or use a different API key

In [ ]:
print("Planning meals for Sharma family...")

try:
    # Call meal planner
    response = await meal_planner_agent.plan_meals(
        family_id="sharma_001",
        request="Quick dinner ideas for tonight"
    )
    
    print(f"\nAgent completed with {len(response)} events")
    
    # Extract and display meaningful agent output
    meal_plan_found = False
    family_prefs_found = False
    
    for i, event in enumerate(response):
        print(f"\n--- Event {i+1} ---")
        
        if hasattr(event, 'content') and event.content:
            if hasattr(event.content, 'parts') and event.content.parts:
                for part in event.content.parts:
                    # Display agent text responses
                    if hasattr(part, 'text') and part.text:
                        text = part.text.strip()
                        if text and len(text) > 10:
                            print(f"Agent Response:")
                            print(f"   {text}")
                            if "meal" in text.lower() or "recipe" in text.lower():
                                meal_plan_found = True
                    
                    # Display function calls and responses
                    elif hasattr(part, 'function_call') and part.function_call:
                        func_name = part.function_call.name
                        print(f"Function Call: {func_name}")
                        if func_name == 'get_family_preferences':
                            print(f"   Getting preferences for family: {part.function_call.args.get('family_id')}")
                        
                    elif hasattr(part, 'function_response') and part.function_response:
                        func_resp = part.function_response.response
                        print(f"Function Response:")
                        
                        # Special handling for family preferences
                        if isinstance(func_resp, dict) and 'preferences' in str(func_resp):
                            family_prefs_found = True
                            prefs = func_resp.get('preferences', {})
                            print(f"   Family: {prefs.get('family_name', 'Unknown')}")
                            print(f"   Members: {prefs.get('members_count', 0)}")
                            print(f"   Dietary: {prefs.get('dietary_restrictions', [])}")
                            print(f"   Cuisines: {prefs.get('preferred_cuisines', [])}")
                        elif isinstance(func_resp, str):
                            # Try to display structured data nicely
                            if len(func_resp) > 200:
                                print(f"   {func_resp[:200]}...")
                            else:
                                print(f"   {func_resp}")
                        else:
                            print(f"   {str(func_resp)[:200]}...")
    
    # Status summary
    print(f"\n=== EXECUTION SUMMARY ===")
    if family_prefs_found:
        print("SUCCESS: Family preferences fetched correctly!")
    else:
        print("Warning: Family preferences not found")
        
    if meal_plan_found:
        print("SUCCESS: Agent generated meal plan content!")
    else:
        print("Warning: No meal plan content detected in agent output")
        
    print(f"Agent execution completed with {len(response)} total events!")

except Exception as e:
    print(f"\nError: {e}")
    if "quota" in str(e).lower():
        print("Rate limit hit - wait 60 seconds and retry")
    elif "api" in str(e).lower():
        print("API issue - check your GOOGLE_API_KEY")

## 4. Demo 2: Natural Language Conversation

**Free-form chat with the orchestrator - Try your own request!**

In [ ]:
# EDIT THIS - Try your own request
my_request = (
    "Suggest easy and fast recepi for monday and tuesday, keep break on wednesday, "
    "thursday if not activity and have time then heavy meal. Kids has activity break on wednesday, "
    "help me find some me time 3-4 hr in week planning, which minnmal impact to all work."
)

print(f"Your request: {my_request}\n")
print("Calling MomsHelperAI orchestrator...\n")

response = await orchestrator.handle_request(
    user_request=my_request,
    family_id='gonzalez_001'
)

print("="*70)
print("AI Response:")
print("="*70)

# Handle orchestrator response structure
if isinstance(response, dict):
    # Display orchestration summary
    agents_executed = response.get('agents_executed', [])
    print(f"Agents executed: {', '.join(agents_executed)}")
    
    # Display meal plan if available
    if response.get('meal_plan'):
        meal_plan = response['meal_plan']
        if isinstance(meal_plan, dict) and 'meal_plan' in meal_plan:
            meals = meal_plan['meal_plan']
            print(f"\nMeal Plan:")
            
            # Handle both list and dict formats
            if isinstance(meals, list):
                print(f"({len(meals)} days planned):")
                for day_plan in meals[:3]:  # Show first 3 days
                    if isinstance(day_plan, dict):
                        day = day_plan.get('day', 'Unknown')
                        print(f"  {day}:")
                        for meal_type in ['breakfast', 'lunch', 'dinner']:
                            meal = day_plan.get(meal_type, {})
                            if meal and isinstance(meal, dict) and meal.get('meal_name'):
                                print(f"    {meal_type.title()}: {meal['meal_name']}")
            elif isinstance(meals, dict):
                print(f"({len(meals)} days planned):")
                for day_name, day_meals in list(meals.items())[:3]:  # Show first 3 days
                    print(f"  {day_name}:")
                    for meal_type in ['breakfast', 'lunch', 'dinner']:
                        meal = day_meals.get(meal_type, {})
                        if meal and isinstance(meal, dict) and meal.get('meal_name'):
                            print(f"    {meal_type.title()}: {meal['meal_name']}")
        
        # Display grocery list summary
        if isinstance(meal_plan, dict) and 'grocery_list' in meal_plan:
            grocery_list = meal_plan['grocery_list']
            if grocery_list:
                total_items = sum(len(items) for items in grocery_list.values() if isinstance(items, list))
                print(f"\nGrocery List: {total_items} items across {len(grocery_list)} categories")
    
    # Display execution summary
    summary = response.get('execution_summary', 'No summary available')
    print(f"\nExecution Summary: {summary}")
    
    # Display any errors
    if response.get('error'):
        print(f"\nError: {response['error']}")
        
elif hasattr(response, 'text'):
    print(response.text)
else:
    print(response)

print("\nReal-time AI response using Google ADK and Gemini")

In [ ]:
# Check final status
print("\n=== FINAL STATUS CHECK ===")
if isinstance(response, dict):
    print(f"✅ Agents executed: {response.get('agents_executed', [])}")
    
    meal_plan = response.get('meal_plan', {})
    has_meals = 'meal_plan' in meal_plan and bool(meal_plan.get('meal_plan'))
    print(f"✅ Has meal data: {has_meals}")
    
    has_grocery = 'grocery_list' in meal_plan and bool(meal_plan.get('grocery_list'))  
    print(f"✅ Has grocery data: {has_grocery}")
    
    weekly_schedule = response.get('weekly_schedule', {})
    has_schedule = bool(weekly_schedule) and weekly_schedule != {'events_count': 7}
    print(f"✅ Has schedule data: {has_schedule}")
    
    shopping_list = response.get('shopping_list', {})
    has_shopping = bool(shopping_list) and shopping_list != {'events_count': 7}
    print(f"✅ Has shopping data: {has_shopping}")
    
    print(f"📋 Summary: {response.get('execution_summary', 'No summary')}")
    
    # Data flow check
    if has_meals and has_grocery:
        print("✅ SUCCESS: MealPlanner → data available for other agents")
    else:
        print("❌ ISSUE: MealPlanner data still missing")
        
    if not has_schedule or not has_shopping:
        print("❌ ISSUE: WeekPlanner or GroceryPlanner not receiving proper data")
    else:
        print("✅ SUCCESS: All agents working with data")
else:
    print("❌ Response not in expected format")
print("=== END FINAL STATUS ===")

In [ ]:
# Test the original problematic request with improved orchestrator
print("=== TESTING ORIGINAL REQUEST WITH FORCED COMPLETION ===")

# The original request that was causing issues
original_request = (
    "Suggest easy and fast recepi for monday and tuesday, keep break on wednesday, "
    "thursday if not activity and have time then heavy meal. Kids has activity break on wednesday, "
    "help me find some me time 3-4 hr in week planning, which minnmal impact to all work."
)

# Test with improved orchestrator
final_response = await orchestrator.handle_request(
    user_request=original_request,
    family_id='gonzalez_001',
    num_days=4  # Monday through Thursday
)

print("=== FINAL RESULTS ===")
if isinstance(final_response, dict):
    print(f"✅ Agents executed: {final_response.get('agents_executed', [])}")
    
    # Check meal plan
    meal_plan = final_response.get('meal_plan', {})
    if 'meal_plan' in meal_plan:
        meals = meal_plan['meal_plan'] 
        print(f"✅ Meal plan: {len(meals)} days generated")
        
        # Check if we have Monday-Thursday meals
        if isinstance(meals, dict):
            days_available = list(meals.keys())
            print(f"✅ Days available: {days_available}")
    
    # Check other outputs
    weekly_schedule = final_response.get('weekly_schedule', {})
    shopping_list = final_response.get('shopping_list', {})
    
    print(f"✅ Weekly schedule: {'Generated' if weekly_schedule and 'events_count' not in weekly_schedule else 'Basic structure'}")
    print(f"✅ Shopping list: {'Generated' if shopping_list and 'events_count' not in shopping_list else 'Basic structure'}")
    
    print(f"📋 Summary: {final_response.get('execution_summary', 'None')}")
    
    print("\\n🎉 ORCHESTRATOR NOW HANDLES LLM STOPPING ISSUES!")
    print("✅ Forced completion prevents workflow breaks")
    print("✅ Emergency fallbacks ensure data flow continues")
    print("✅ All downstream agents receive proper data")

print("=== END ORIGINAL REQUEST TEST ===")

In [ ]:
# Check the response structure and data flow
print("=== DEBUG ORCHESTRATOR RESPONSE ===")
print(f"Response type: {type(response)}")
print(f"Response keys: {list(response.keys()) if isinstance(response, dict) else 'Not a dict'}")

if isinstance(response, dict):
    print(f"\nAgents executed: {response.get('agents_executed', [])}")
    
    meal_plan = response.get('meal_plan', {})
    print(f"\nMeal plan type: {type(meal_plan)}")
    print(f"Meal plan keys: {list(meal_plan.keys()) if isinstance(meal_plan, dict) else 'Not a dict'}")
    
    if isinstance(meal_plan, dict) and 'meal_plan' in meal_plan:
        meals = meal_plan['meal_plan']
        print(f"Inner meal_plan type: {type(meals)}")
        if isinstance(meals, list):
            print(f"Number of days: {len(meals)}")
        elif isinstance(meals, dict):
            print(f"Meal plan dict keys (days): {list(meals.keys())}")
    
    weekly_schedule = response.get('weekly_schedule', {})
    print(f"\nWeekly schedule keys: {list(weekly_schedule.keys()) if isinstance(weekly_schedule, dict) else 'None'}")
    
    shopping_list = response.get('shopping_list', {})
    print(f"Shopping list keys: {list(shopping_list.keys()) if isinstance(shopping_list, dict) else 'None'}")
    
    print(f"\nExecution summary: {response.get('execution_summary', 'None')}")
    if 'error' in response:
        print(f"Error: {response['error']}")

In [ ]:
# Let's try running the MealPlanner alone to see its actual response
print("=== TESTING MEALPLANNER DIRECTLY ===")

test_response = await meal_planner_agent.plan_meals(
    family_id="gonzalez_001",
    request="Plan dinner for today"
)

print(f"MealPlanner response type: {type(test_response)}")
print(f"Number of events: {len(test_response) if isinstance(test_response, list) else 'Not a list'}")

# Let's examine the actual response structure
if isinstance(test_response, list):
    for i, event in enumerate(test_response):
        print(f"\n--- Event {i+1} ---")
        if hasattr(event, 'content') and event.content:
            if hasattr(event.content, 'parts') and event.content.parts:
                for j, part in enumerate(event.content.parts):
                    if hasattr(part, 'function_call') and part.function_call:
                        func_name = part.function_call.name
                        print(f"Function Call: {func_name}")
                        if func_name == 'save_meal_plan':
                            print(f"  Args keys: {list(part.function_call.args.__dict__.keys()) if hasattr(part.function_call.args, '__dict__') else 'No args'}")
                    elif hasattr(part, 'function_response') and part.function_response:
                        resp = part.function_response.response
                        print(f"Function Response: {type(resp)}")
                        if isinstance(resp, dict):
                            print(f"  Response keys: {list(resp.keys())}")
                            if 'plan_id' in resp:
                                print(f"  Plan ID: {resp['plan_id']}")
                    elif hasattr(part, 'text') and part.text:
                        text_snippet = part.text.strip()[:100]
                        if text_snippet:
                            print(f"Text: {text_snippet}...")
print("\n=== END MEALPLANNER TEST ===")

In [ ]:
# Test the improved orchestrator
print("=== TESTING IMPROVED ORCHESTRATOR ===")

# Reload the orchestrator module to get latest changes
import importlib
importlib.reload(sys.modules['agents.orchestrator'])
from agents.orchestrator import orchestrator

# Test with a simpler, more direct request
simple_request = "Plan meals for this week"

improved_response = await orchestrator.handle_request(
    user_request=simple_request,
    family_id='gonzalez_001',
    num_days=3  # Shorter to avoid rate limits
)

print(f"Improved response type: {type(improved_response)}")
print(f"Improved response keys: {list(improved_response.keys()) if isinstance(improved_response, dict) else 'Not a dict'}")

if isinstance(improved_response, dict):
    meal_plan = improved_response.get('meal_plan', {})
    print(f"Meal plan status: {meal_plan.get('status', 'No status')}")
    print(f"Meal plan keys: {list(meal_plan.keys())}")
    
    if 'meal_plan' in meal_plan and meal_plan['meal_plan']:
        inner_plan = meal_plan['meal_plan']
        print(f"Inner meal plan type: {type(inner_plan)}")
        if isinstance(inner_plan, dict):
            print(f"Days available: {list(inner_plan.keys())}")
        elif isinstance(inner_plan, list):
            print(f"Number of days: {len(inner_plan)}")

print("=== END IMPROVED TEST ===")

In [ ]:
# Quick check of the latest improved_response
print("=== QUICK STATUS CHECK ===")
if 'improved_response' in locals():
    print(f"Response type: {type(improved_response)}")
    if isinstance(improved_response, dict):
        print(f"Agents executed: {improved_response.get('agents_executed', [])}")
        meal_plan = improved_response.get('meal_plan', {})
        print(f"Meal plan status: {meal_plan.get('status', 'unknown')}")
        print(f"Has meal_plan data: {'meal_plan' in meal_plan and bool(meal_plan.get('meal_plan'))}")
        print(f"Has grocery_list: {'grocery_list' in meal_plan and bool(meal_plan.get('grocery_list'))}")
        print(f"Summary: {meal_plan.get('summary', 'No summary')}")
else:
    print("No improved_response variable found")
print("=== END STATUS ===")

## Demo 3: Human-in-the-Loop Workflow ✋

This demonstration shows how to add human approval checkpoints to the multi-agent workflow.

### Step 3.1: Configure Approval Callback

We'll create a callback function that pauses the workflow after meal planning:

In [ ]:
# Step 1: Create approval callback that pauses after MealPlanner
meal_plan_output = {}

def approval_callback(agent_name: str, output: dict) -> bool:
    """
    Callback function invoked after MealPlanner completes.
    Returns False to pause workflow for human review.
    """
    global meal_plan_output
    print(f"\n{agent_name} has completed. Pausing for human review...")
    meal_plan_output = output  # Store output for review
    return False  # Return False to pause workflow

# Define request for testing
user_request = "Plan dinner for this week"

# Call orchestrator with approval callback
result = await orchestrator.handle_request(
    user_request=user_request,
    family_id="sharma_001", 
    approval_callback=approval_callback  # Add callback parameter
)

print("\nWorkflow paused after meal planning")
print(f"Status: {result.get('status', 'unknown')}")

### Step 3.2: Review Meal Plan

Examine the generated meal plan before approving:

In [ ]:
# Step 2: Display meal plan for human review
import json

print("📋 MEAL PLAN FOR REVIEW:\n")
print("=" * 60)
if meal_plan_output:
    print(json.dumps(meal_plan_output, indent=2))
else:
    print("No meal plan available. Run Step 1 first.")
print("=" * 60)

### Step 3.3: Human Decision

Make your approval decision:

In [ ]:
# Step 3: Get human approval
approval = input("\n👤 Do you approve this meal plan? (yes/no): ").strip().lower()

if approval == "yes":
    print("✅ Meal plan approved! Proceeding to next agents...")
    human_approved = True
else:
    print("❌ Meal plan rejected. Workflow will not continue.")
    human_approved = False

### Step 3.4: Continue or Stop Workflow

Based on approval, either continue with remaining agents or stop:

In [ ]:
# Step 4: Continue workflow if approved
if human_approved:
    print("\n🔄 Continuing with remaining agents...")
    
    # Manually call remaining agents
    from agents.week_planner import WeekPlannerAgent
    from agents.grocery_planner import GroceryPlannerAgent
    
    # Week planning
    week_planner = WeekPlannerAgent(storage)
    schedule = week_planner.handle_request(family_data, meal_plan_output)
    
    # Grocery planning  
    grocery_planner = GroceryPlannerAgent(storage)
    grocery_list = grocery_planner.handle_request(family_data, meal_plan_output)
    
    print("\n✅ COMPLETE WORKFLOW FINISHED!")
    print(f"📅 Schedule created: {len(schedule.get('events', []))} events")
    print(f"🛒 Grocery items: {len(grocery_list.get('items', []))}")
else:
    print("\n⏹️ Workflow stopped. No further agents executed.")

### 🎯 Key Takeaway

**Human-in-the-Loop (HITL)** adds approval checkpoints to your multi-agent workflow:

- ✅ **Backward Compatible**: Existing code works without changes
- ✅ **Optional Parameter**: `approval_callback` defaults to `None`  
- ✅ **Flexible Control**: Return `False` to pause, `True` to continue
- ✅ **Zero Breaking Changes**: All existing functionality preserved

This pattern gives you control over critical decision points while maintaining the benefits of automated multi-agent coordination.

==============================================================================

## Conclusion

This notebook demonstrated **MomsHelperAI** - a production-ready multi-agent system using:

- **Google Agent Development Kit (ADK)**
- **Gemini 2.5 Flash Lite LLM** (Real AI responses)
- **Multi-agent architecture** with specialized sub-agents
- **SQLite** for data persistence
- **Family context** - culturally aware meal planning and activities

==============================================================================

**Built using Google Agent Development Kit (ADK)**